# **Exercise 2.4: Evaluating Hyperparameters. Deep Learning**

## 1. Importing Libraries and Data

In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [6]:
path = r'C:\Users\Nuria Miquel\Desktop\nuria\Machine Learning'

In [8]:
df = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'weather_cleaned2.csv'), index_col = False)
pleasant = pd.read_csv(os.path.join(path, 'Data', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'), index_col = False)

In [10]:
df.head()

,DATE,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,19600102,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,19600103,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,19600104,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,19600105,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [12]:
pleasant.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2. Data Wrangling

In [59]:
df.drop('DATE', inplace = True)
pleasant.drop('DATE', axis = 1, inplace = True)

KeyError: "['DATE'] not found in axis"

In [47]:
df.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [57]:
pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
print(df.shape)
print(pleasant.shape)

(22950, 135)
(22950, 15)


## 3. Reshape

In [186]:
X = np.array(df)
y = np.array(pleasant)

In [188]:
X = X.reshape(-1,15,9)

In [190]:
y =  np.argmax(y, axis = 1)

In [192]:
print(X.shape)
print(y.shape)

(22950, 15, 9)
(22950,)


In [194]:
# The Bayesian optimization function only accepts y data in “multiclass” and “binary” layouts but not in “multilabel-indicator.”
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

## 4. Data Split

In [222]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [224]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


## 5. Bayesian optimization

In [97]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [99]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [131]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (100, 500), #(10, 50), #
    'epochs':(20, 40),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/23
29/29 - 5s - 160ms/step - accuracy: 0.6255 - loss: 2.7018
Epoch 2/23
29/29 - 1s - 27ms/step - accuracy: 0.6440 - loss: 2.6959
Epoch 3/23
29/29 - 1s - 26ms/step - accuracy: 0.6440 - loss: 2.6909
Epoch 4/23
29/29 - 1s - 25ms/step - accuracy: 0.6440 - loss: 2.6867
Epoch 5/23
29/29 - 1s - 25ms/step - accuracy: 0.6440 - loss: 2.6831
Epoch 6/23
29/29 - 1s - 25ms/step - accuracy: 0.6440 - loss: 2.6797
Epoch 7/23
29/29 - 1s - 24ms/step - accuracy: 0.6440 - loss: 2.6766
Epoch 8/23
29/29 - 1s - 26ms/step - accuracy: 0.6440 - loss: 2.6737
Epoch 9/23
29/29 - 1s - 26ms/step - accuracy: 0.6440 - loss: 2.6708
Epoch 10/23
29/29 - 1s - 27ms/step - accuracy: 

ValueError: Input y contains NaN.

In [145]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 230,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 38,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x245c6f28740>}

## 6. Running CNN with Optimized Parameters

In [203]:
# Set the model with optimized hyperparameters

epochs = 38
batch_size = 230

timesteps = len(X_train[0])  
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  
neurons = 61
normalization = 0.770967179954561  
dropout = 0.7296061783380641  
dropout_rate = 0.19126724140656393  
optimizer = Adadelta(learning_rate=0.7631771981307285) 

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

# Add second set of dense layers
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())  # Max-pooling layer to downsample the data
model.add(Flatten())  # Flatten for the dense layers
model.add(Dense(n_classes, activation='softmax'))  # Output layer for classification

# Compile the model with specified loss and optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [205]:
model.summary()

Model: "sequential_76"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_76 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_384 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_385 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_386 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_76 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_76 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_387 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [207]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/38
75/75 - 3s - 45ms/step - accuracy: 0.6682 - loss: 1.1192
Epoch 2/38
75/75 - 1s - 13ms/step - accuracy: 0.7253 - loss: 0.8134
Epoch 3/38
75/75 - 1s - 12ms/step - accuracy: 0.7519 - loss: 0.7304
Epoch 4/38
75/75 - 1s - 13ms/step - accuracy: 0.7671 - loss: 0.6807
Epoch 5/38
75/75 - 1s - 13ms/step - accuracy: 0.7821 - loss: 0.6329
Epoch 6/38
75/75 - 1s - 13ms/step - accuracy: 0.7950 - loss: 0.6015
Epoch 7/38
75/75 - 1s - 13ms/step - accuracy: 0.8057 - loss: 0.5688
Epoch 8/38
75/75 - 1s - 14ms/step - accuracy: 0.8141 - loss: 0.5349
Epoch 9/38
75/75 - 1s - 13ms/step - accuracy: 0.8234 - loss: 0.5107
Epoch 10/38
75/75 - 1s - 13ms/step - accuracy: 0.8285 - loss: 0.4964
Epoch 11/38
75/75 - 1s - 13ms/step - accuracy: 0.8408 - loss: 0.4652
Epoch 12/38
75/75 - 1s - 14ms/step - accuracy: 0.8431 - loss: 0.4544
Epoch 13/38
75/75 - 1s - 14ms/step - accuracy: 0.8509 - loss: 0.4341
Epoch 14/38
75/75 - 1s - 14ms/step - accuracy: 0.8573 - loss: 0.4152
Epoch 15/38
75/75 - 1s - 12ms/step - accura

## 7. Confusion Matrix

In [210]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [248]:
# Predict the class probabilities
y_pred = model.predict(X_test)

# Convert y_test and y_pred to class labels
if y_test_one_hot.ndim == 1:
    y_test_labels = y_test_one_hot
else:
    y_test_labels = np.argmax(y_test_one_hot, axis=1)

if y_pred.ndim == 1:
    y_pred_labels = y_pred
else:
    y_pred_labels = np.argmax(y_pred, axis=1)

# Manually calculate accuracy
correct_predictions = np.sum(y_test_labels == y_pred_labels)
total_samples = len(y_test_labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 91.50%


In [240]:
# Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

In [242]:
# Evaluate
print(confusion_matrix(y_test_one_hot, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3556        73        12      10           5         5       0   
BELGRADE       75       983        12       4           0         3       0   
BUDAPEST       22        16       155       6           1         4       0   
DEBILT          7         4         9      58           2         2       0   
DUSSELDORF      1         0         1       2          12         7       0   
HEATHROW       12         1         0       1           2        36       0   
KASSEL          1         1         1       0           2         0       3   
LJUBLJANA       6         1         3       0           0         0       0   
MAASTRICHT      6         0         0       0           0         0       0   
MADRID         41         3         9       0           0         1       0   
MUNCHENB   